---

# CSCI 3202, Spring 2024
# Homework 4
# Due: Friday April 5, 2024 at 5:59 PM

<br> 

### Your name: Andrew Logue

<br> 

---

Some useful packages and libraries:



In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import colors
from collections import deque
import heapq
import unittest
from scipy import stats
import copy as cp
from time import time

---

## Problem 1: Game Theory - Playing "intelligent" Tic-Tac-Toe

<img src="https://www.cookieshq.co.uk/images/2016/06/01/tic-tac-toe.png" width="150"/>



### (1a)   Defining the Tic-Tac-Toe class structure

Fill in this class structure for Tic-Tac-Toe using what we did during class as a guide.
* `moves` is a list of tuples to represent which moves are available. Recall that we are using matrix notation for this, where the upper-left corner of the board, for example, is represented at (1,1).
* `result(self, move, state)` returns a ***hypothetical*** resulting `State` object if `move` is made when the game is in the current `state`
* `compute_utility(self, move, state)` calculates the utility of `state` that would result if `move` is made when the game is in the current `state`. This is where you want to check to see if anyone has gotten `nwin` in a row
* `game_over(self, state)` - this wasn't a method, but it should be - it's a piece of code we need to execute repeatedly and giving it a name makes clear what task the piece of code performs. Returns `True` if the game in the given `state` has reached a terminal state, and `False` otherwise.
* `utility(self, state, player)` also wasn't a method earlier, but also should be.  Returns the utility of the current state if the player is X and $-1 \cdot$ utility if the player is O.
* `display(self)` is a method to display the current game `state`, You get it for free! because this would be super frustrating without it.
* `play_game(self, player1, player2)` returns an integer that is the utility of the outcome of the game (+1 if X wins, 0 if draw, -1 if O wins). `player1` and `player2` are functional arguments that we will deal with in parts **1b** and **1d**.

Some notes:
* Assume X always goes first.
* Do **not** hard-code for 3x3 boards.
* You may add attributes and methods to these classes as needed for this problem.

In [6]:
class State:
    def __init__(self, moves):
        self.to_move = 'X'
        self.utility = 0
        self.board = {}
        self.moves = cp.copy(moves)

        
class TicTacToe:
    
    def __init__(self, nrow=3, ncol=3, nwin=3, nexp=0):
        self.nrow = nrow
        self.ncol = ncol
        self.nwin = nwin
        moves = [(row, col) for row in range(1, nrow + 1) for col in range(1, ncol + 1)] # insert your general list of nrow x ncol moves here
        self.state = State(moves)
        self.nexp = nexp

    def result(self, move, state):
        '''
        What is the hypothetical result of move `move` in state `state` ?
        move  = (row, col) tuple where player will put their mark (X or O)
        state = a `State` object, to represent whose turn it is and form
                the basis for generating a **hypothetical** updated state
                that will result from making the given `move`
        '''

        # First, create a copy of the given state. We use `deepcopy` so
        # that we do not prematurely tamper with the actual current 
        # Tic-Tac-Toe state, since we mostly use this method
        # to get hypothetical results as we search our game tree, not
        # for actual moves.
        new_state = cp.deepcopy(state)
        new_state.board[move] = state.to_move
        new_state.moves.remove(move)
        new_state.to_move = 'O' if state.to_move=='X' else 'X'
        new_state.utility = self.compute_utility(move, state)
        return new_state
        

        
    def compute_utility(self, move, state):
        '''
        What is the utility of making move `move` in state `state`?
        If 'X' wins with this move, return 1;
        if 'O' wins return -1;
        else return 0.
        '''        

        row, col = move
        player = state.to_move
        
        # create a hypothetical copy of the board, with 'move' executed
        board = cp.deepcopy(state.board)
        board[move] = player

        # what are all the ways 'player' could with with 'move'?
        
        # check for row-wise win
        in_a_row = 0
        for c in range(1,self.ncol+1):
            in_a_row += board.get((row,c))==player

        # check for column-wise win
        in_a_col = 0
        for r in range(1,self.nrow+1):
            in_a_col += board.get((r,col))==player

        # check for NW->SE diagonal win
        in_a_diag1 = 0
        for r in range(row,0,-1):
            in_a_diag1 += board.get((r,col-(row-r)))==player
        for r in range(row+1,self.nrow+1):
            in_a_diag1 += board.get((r,col-(row-r)))==player

        # check for SW->NE diagonal win
        in_a_diag2 = 0
        for r in range(row,0,-1):
            in_a_diag2 += board.get((r,col+(row-r)))==player
        for r in range(row+1,self.nrow+1):
            in_a_diag2 += board.get((r,col+(row-r)))==player
        
        if self.nwin in [in_a_row, in_a_col, in_a_diag1, in_a_diag2]:
            return 1 if player=='X' else -1
        else:
            return 0

        

    def game_over(self, state):
        '''game is over if someone has won (utility!=0) or there
        are no more moves left'''

        return state.utility!=0 or len(state.moves)==0
   

    
    def utility(self, state, player):
        '''Return the value to player; 1 for win, -1 for loss, 0 otherwise.'''

        return state.utility if player=='X' else -state.utility
      
               
    def display(self):
        for row in range(1, self.nrow+1):
            for col in range(1, self.ncol+1):
                print(self.state.board.get((row, col), '.'), end=' ')
            print()
        
    def play_game(self, player1, player2):
        '''Play a game of tic-tac-toe!'''

        turn_limit = self.nrow*self.ncol  # limit in case of buggy code
        turn = 0
        while turn<=turn_limit:
            for player in [player1, player2]:
                turn += 1
                move = player(self)
                self.state = self.result(move, self.state)
                if self.game_over(self.state):
                    # self.display()
                    return self.state.utility 
 

### (1b) Define a random player

Define a function `random_player` that takes a single argument of the `TicTacToe` class and returns a random move out of the available legal moves in the `state` of the `TicTacToe` game.

In your code for the `play_game` method above, make sure that `random_player` could be a viable input for the `player1` and/or `player2` arguments.

In [7]:
def random_player(game):
    '''A player that chooses a legal move at random out of all
    available legal moves in Tic-Tac-Toe state argument'''

    possible_actions = game.state.moves
    random_move = possible_actions[np.random.randint(low=0, high=len(possible_actions))]
    
    return random_move
    


We know from experience and/or because I'm telling you right now that if two `random_player`s play many games of Tic-Tac-Toe against one another, whoever goes first will win about 58% of the time.  Verify that this is the case by playing at least 1,000 games between two random players. Report the proportion of the games that the first player has won.

**"Unit tests":** If you are wondering how close is close enough to 58%, I simulated 100 tournaments of 1,000 games each. The min-max range of win percentage by the first player was 54-63%.

In [9]:
niter = 1000
wins = 0
draws = 0
losses = 0
for k in range(niter):
    out = TicTacToe().play_game(random_player, random_player)
    if out==1:
        wins += 1
    elif out==-1:
        losses += 1
    else:
        draws += 1
        
print('First-player winning percentage = {}'.format(wins/niter))

First-player winning percentage = 0.581


### (1c) What about playing randomly on different-sized boards?

What does the long-term win percentage appear to be for the first player in a 4x4 Tic-Tac-Toe tournament, where 4 marks must be connected for a win?  Support your answer using a simulation and printed output, similar to **1b**.

**Also:** The win percentage should have changed substantially. Did the decrease in wins turn into more losses for the first player or more draws? Write a few sentences explaining the behavior you observed.  *Hint: think about how the size of the state space has changed.*

In [10]:
niter = 1000
wins = 0
draws = 0
losses = 0
for k in range(niter):
    out = TicTacToe(4,4,4).play_game(random_player, random_player)
    if out==1:
        wins += 1
    elif out==-1:
        losses += 1
    else:
        draws += 1
        
print('First-player winning percentage = {}'.format(wins/niter))

First-player winning percentage = 0.313


### (1d) Define an alpha-beta player

Alright. Let's finally get serious about our Tic-Tac-Toe game.  No more fooling around!

Craft a function called `alphabeta_player` that takes a single argument of a `TicTacToe` class object and returns the minimax move in the `state` of the `TicTacToe` game. As the name implies, this player should be implementing alpha-beta pruning as described in the textbook and lecture.

Note that your alpha-beta search for the minimax move should include function definitions for `max_value` and `min_value` (see the aggressively realistic pseudocode from the lecture slides).

In your code for the `play_game` method above, make sure that `alphabeta_player` could be a viable input for the `player1` and/or `player2` arguments.

In [22]:
def alphabeta_player(game):
    """A player that chooses the best move using the minimax algorithm with alpha-beta pruning."""
    alpha = float("-inf")
    beta = float("inf")
    _, move = max_value(game, alpha, beta)
    return move

def max_value(game, alpha, beta):
    """Return the utility value of the best move and the corresponding move for maximizing player."""
    if game.game_over(game.state):
        return game.utility(game.state, 'X'), None

    v = float("-inf")
    best_move = None
    for move in game.state.moves:
        new_state = game.result(move, game.state)
        min_val, _ = min_value(game, alpha, beta)
        if min_val > v:
            v = min_val
            best_move = move
        if v >= beta:
            return v, best_move
        alpha = max(alpha, v)
    return v, best_move

def min_value(game, alpha, beta):
    """Return the utility value of the best move and the corresponding move for minimizing player."""
    if game.game_over(game.state):
        return game.utility(game.state, 'X'), None

    v = float("inf")
    best_move = None
    for move in game.state.moves:
        new_state = game.result(move, game.state)
        max_val, _ = max_value(game, alpha, beta)
        if max_val < v:
            v = max_val
            best_move = move
        if v <= alpha:
            return v, best_move
        beta = min(beta, v)
    return v, best_move

Run the following tests, using a standard 3x3 Tic-Tac-Toe board. Run **10 games for each test**, and track the number of wins, draws and losses. Report these results for each case.

1. An alpha-beta player who plays first versus a random player who plays second.
2. Two alpha-beta players.

**Nota bene:** Test your code with fewer games between the players to start, because the alpha-beta player will require substantially more compute time than the random player.  This is why I only ask for 10 games, which still might take a minute or two. You are welcome to run more than 10 tests if you'd like. 

In [23]:
# An alpha-beta player who plays first versus a random player who plays second.
niter = 10
wins = 0
draws = 0
losses = 0
for k in range(niter):
    out = TicTacToe().play_game(alphabeta_player, random_player)
    if out==1:
        wins += 1
    elif out==-1:
        losses += 1
    else:
        draws += 1
        
print('A-B Vs. random winning percentage = ', [wins, losses, draws])

# Two alpha-beta players.
niter = 10
wins = 0
draws = 0
losses = 0
for k in range(niter):
    out = TicTacToe().play_game(alphabeta_player, alphabeta_player)
    if out==1:
        wins += 1
    elif out==-1:
        losses += 1
    else:
        draws += 1
        
print('A-B Vs. A-B winning percentage = ', [wins, losses, draws])

RecursionError: maximum recursion depth exceeded while calling a Python object

### (1e) What has pruning ever done for us?

Calculate the number of number of states expanded by the minimax algorithm, **with and without pruning**, to determine the minimax decision from the initial 3x3 Tic-Tac-Toe board state.  This can be done in many ways, but writing out all the states by hand is **not** one of them (as you will find out!).

Then compute the percent savings that you get by using alpha-beta pruning. i.e. Compute $\frac{\text{Number of nodes expanded with pruning}}{\text{Number of nodes expanded with minimax}} $

Write a sentence or two, commenting on the difference in number of nodes expanded by each search.

In [ ]:
# Your code here.

---


## Problem 2:  Bayesian network to model heart disease

The following Bayesian network is based loosely on a study that examined heart disease risk factors in 167 elderly individuals in South Carolina.  Note that this figure uses Y and N to represent Yes and No, whereas in class we used the equivalent T and F to represent True and False Boolean values.

<img src="http://www.cs.colorado.edu/~tonyewong/home/resources/hw05_bayesnet_heartdisease.png" style="width: 650px;"/>

### (2a) 

Create a `BayesNet` object to model this.  Below is the code for the (conditional) probability `P` function and `BayesNode` class as well, that we used in our coding notebook to represent the variable nodes and calculate probabilities. You can code this however you want, subject to the following constraints:
1. the nodes are represented using the `BayesNode` class and can work with the `P` function for probabilities,
1. your `BayesNet` structure keeps track of which nodes are in the Bayes net, as well as
1. which nodes are the parents/children of which other nodes.

Some *suggested* skeleton codes for a class structure are given. The suggestions for methods to implement are in view of the fact that we will need to calculate some probabilities, which is going to require us to `find_node`s and `find_values` that nodes can take on.

In [28]:
## For the sake of brevity...
T, F = True, False

## From class:
def P(var, value, evidence={}):
    '''The probability distribution for P(var | evidence), 
    when all parent variables are known (in evidence)'''
    if len(var.parents)==1:
        # only one parent
        row = evidence[var.parents[0]]
    else:
        # multiple parents
        row = tuple(evidence[parent] for parent in var.parents)
    return var.cpt[row] if value else 1-var.cpt[row]

## Also from class:
class BayesNode:
    
    def __init__(self, name, parents, values, cpt):
        if isinstance(parents, str):
            parents = parents.split()
            
        if len(parents)==0:
            # if no parents, empty dict key for cpt
            cpt = {(): cpt}
        elif isinstance(cpt, dict):
            # if there is only one parent, only one tuple argument
            if cpt and isinstance(list(cpt.keys())[0], bool):
                cpt = {(v): p for v, p in cpt.items()}

        self.variable = name
        self.parents = parents
        self.cpt = cpt
        self.values = values
        self.children = []
        
    def __repr__(self):
        return repr((self.variable, ' '.join(self.parents)))    

    
##===============================================##
## Suggested skeleton codes for a BayesNet class ##
##===============================================##

class BayesNet:
    '''Bayesian network containing only boolean-variable nodes.'''

    def __init__(self, nodes):
        '''Initialize the Bayes net by adding each of the nodes,
        which should be a list BayesNode class objects ordered
        from parents to children (`top` to `bottom`, from causes
        to effects)'''
        
        self.variables = {node.variable: node for node in self.nodes}
        for node in self.nodes:
            for parent in node.parents:
                self.find_node(parent).children.append(node)

                
    def add(self, node):
        '''Add a new BayesNode to the BayesNet. The parents should all
        already be in the net, and the variable itself should not be'''
        assert node.variable not in self.variables
        assert all((parent in self.variables) for parent in node.parents)
        
        self.nodes.append(node)
        self.variables[node.variable] = node
        for parent in node.parents:
            self.find_node(parent).children.append(node)
        

            
    def find_node(self, var):
        '''Find and return the BayesNode in the net with name `var`'''
        
        return self.variables[var]


        
    def find_values(self, var):
        '''Return the set of possible values for variable `var`'''
        
        return self.find_node(var).values
        

    
    def __repr__(self):
        return 'BayesNet({})'.format(self.nodes)

In [29]:
# Define the nodes

Sm = BayesNode('smoking and alcohol', [], [T, F], 0.2)
ME = BayesNode('moderate exercise', [], [T, F], 0.5)
HBP = BayesNode('high blood pressure', [Sm, ME], [T, F], 0.1)
Ath = BayesNode('moderate exercise', [], [T, F], 0.53)
FH = BayesNode('moderate exercise', [], [T, F], 0.15)
HD = BayesNode('moderate exercise', [HBP, Ath, FH], [T, F], 0.5)
Ang = BayesNode('moderate exercise', [HD], [T, F], 0.15)
Rapid = BayesNode('moderate exercise', [HD], [T, F], 0.15)

# Create a Bayes net with those nodes and connections

bayes_net = BayesNet([Sm, ME, HBP, Ath, FH, HD, Ang, Rapid])

AttributeError: 'BayesNet' object has no attribute 'nodes'

### (2b)

Craft a function `get_prob(X, e, bn)` to return the **normalized** probability distribution of variable `X` in Bayes net `bn`, given the evidence `e`.  That is, return $P(X \mid e)$. The arguments are:
* `X` is some representation of the variable you are querying the probability distribution of. Either a string (the variable name from the `BayesNode` or a `BayesNode` object itself are good options.
* `e` is some representation of the evidence your probability is conditioned on. When given an empty argument (or `None`) for `e`, `get_prob` should return the marginal distribution $P(X)$.
* `bn` is your `BayesNet` object.

You may do this using the `enumeration` algorithm from class (pseudocode is in the book), or by brute force (i.e., use a few `for` loops). Either way, you should be using your `BayesNet` object to keep track of all the nodes and relationships between nodes so your `get_prob` function knows these things.

In [ ]:
# Your code here.

Use your `get_prob` function to calculate the following probabilities. Print them to the screen and compare to the original Bayes net figure given to make sure the output passes these "unit tests".

1. The marginal probability of `Family History` is $P(FH=T)=0.15$
2. The probability of *not* experiencing `Angina Pectoris`, given `Heart Disease` is observed, is $P(Ang=F \mid HD=T)=1-0.85=0.15$
3. The probability of `High Blood Pressure`, given a person does `Smoke and/or use Alcohol` but does not get `Moderate Exercise`, is $P(HBP=T \mid Sm=T, ME=F)=0.72$

In [ ]:
# Your code here.

### (2c)

Calculate the probability of observing someone with `High Blood Pressure`, $P(HBP=T)$, *by hand*, showing all work in Markdown/LateX below.

**Verify** your calculation using your `get_prob` function.

In [ ]:
# Your code here.

### (2d)

Now calculate the following probabilities using your `get_prob` function.

[i] The probability of an arbitrary individual having `Heart Disease`, $P(HD=T)$

In [ ]:
# Your code here.

[ii] The probability that an individual does *not* have `Heart Disease`, given that `Rapid Heartbeat` was observed, $P(HD=F \mid Rapid=T)$

In [ ]:
# Your code here.

[iii] The probability of an individual having `High Blood Pressure` if they have `Heart Disease` and a `Family History`, $P(HBP=T \mid HD=T, FH=T)$

In [ ]:
# Your code here.

[iv] The probability that an individual is a `Smoker/Alcohol User` if they have `Heart Disease`, $P(Sm=T \mid HD=T)$

In [ ]:
# Your code here.

[v] How would you expect the probability in [iv] to change if you also know the individual has `High Blood Pressure`?  Verify your hypothesis by calculating the relevant probability.

In [ ]:
# Your code here.

[vi] How would you expect the probability in [v] to change if you also know that the individual does *not* get `Moderate Exercise` (in addition to having `Heart Disease` and `High Blood Pressure`)?  Explain your answer using concepts from class.  Verify your answer by calculating the relevant probability.

In [ ]:
# Your code here.

---


## Problem 3:  Bayesian network to model decision-making

Let's consider using a Bayesian network to model our decision about whether or not to ride our bike to work today.  This decision depends heavily on the weather, so let's focus on that.

In class, we focused on Boolean variables.  For example, we might base our biking decision on whether or not it is raining.  But in reality, it probably matters *how hard* it is raining.  So suppose we break the variable `Rain` up into three discrete bins: `none`, `light` and `heavy`.

The temperature also factors into our decision.  There is definitely a sweet spot, where temperatures are neither too warm nor too cold, so it is very likely we would enjoy riding our bike.  So we can model the variable `Temperature` also using three discrete bins: `cold`, `moderate` and `warm`.

So a Bayesian network to model our decision for whether or not to bike to work could be as follows, where the first letter of each discrete bin is used to denote that variable value (i.e., `R=h` stands for heavy rain conditions).

<img src="http://www.cs.colorado.edu/~tonyewong/home/resources/bayesnet_biking2.png" style="width: 650px;"/>

### (3a)

Modify the `P` probability function to be able to handle these ternary parent nodes.

In [ ]:
# Your code here.

Set up `BayesNode` objects for each of `Rain`, `Temp` and `Bike`, and create a `BayesNet` object to model the Bayesian network for this decision.  Again, you can use whatever structure you wish for your `BayesNet`, but please use the `BayesNode` class.  You may need to make minor modifications to the `BayesNode` class (e.g., changing/adding attributes), although none are strictly necessary.

In [ ]:
# Your code here.

**Verify** that your modified probability function `P` is working by checking the following "unit tests". Print the output to screen and compare to what you expect from the figure above.

1. The marginal probability of no rain is $P(Rain=n)=0.8$
1. The marginal probability of light rain is $P(Rain=l)=0.15$
1. The marginal probability of heavy rain is $P(Rain=h)=0.05$
1. The probability of biking given that it is raining heavily and the temperature is cold, is $P(Bike=T \mid Rain=h, Temp=c)=0.2$

In [ ]:
# Your code here.

### (3b)

Make any necessary modifications to your `get_prob` function from Problem 2, so that you can use it to calculate marginal probabilities and conditional probabilities for this problem. It is possible that your function does not require any modifications.

Use `get_prob` to calculate $P(Bike)$, the probability distribution for whether or not you will ride your bike on any given day.

In [ ]:
# Your code here.

Use `get_prob` to calculate the probability that you will ride your bike, given that it is lightly raining.

In [ ]:
# Your code here.

### (3c)

We are trapped indoors because some jerk gave us a ton of Intro to Artificial Intelligence homework to do.  Suppose we look out the window and see people biking. They sure do look like they're having fun! *Given* this information, we can actually make inferences regarding the temperature outside!  What is the probability distribution for temperature, given that we observe people biking?

First, compute this using your `get_prob` function.

In [ ]:
# Your code here.

### (3d)

Confirm your answer to **3c** by hand, showing *all* relevant work below in a LateX/Markdown cell.